In [1]:
import ee
import geemap

In [2]:
ee.Initialize(project='nb-lidar')

In [3]:
# DEFINE CONSTANTS
folder = "exp4"
aoi_id = "projects/nb-lidar/assets/aoi_nb_south"
input_data_id = "projects/nb-lidar/assets/training_data/wetland_only"
terrain_type = "dsm"

In [4]:
aoi = ee.FeatureCollection(aoi_id).geometry()
input_data = ee.FeatureCollection(input_data_id)

In [5]:
from src import image_processing as proc

s1 = proc.fetch_and_proecss_s1_seasonal(aoi)
s2 = proc.fetch_and_proeces_s2_sr_seasonal(aoi)
al = proc.fetch_and_process_alos()
ft = proc.fetch_fourier_transform()
dsm = proc.fetch_dsm_ta()
dtm = proc.fetch_dtm_ta()
chm = dtm.select('Elevation').subtract(dsm.select('Elevation')).rename('CHM')
stack = ee.Image.cat(s1, s2, dsm, dtm, chm)

In [6]:
stack.bandNames()

In [7]:
# create a wetland mask
wetland_mask = ee.Image('projects/nb-lidar/assets/exp3_dtm_classification').eq(3) # 3 = wetland class value

In [8]:
# apply mask to the stack
masked_stack = stack.updateMask(wetland_mask)

In [9]:
# Extract Features
features = masked_stack.sampleRegions(
    collection=input_data,
    tileScale=16,
    scale=10
)

In [10]:
# split features into train and test
train = features.filter('random < 0.7')
test = features.filter('random >= 0.7')

In [11]:
# Train the Random Forest Model
rf = ee.classifier.Classifier.smileRandomForest(1000).train(train, 'class_name', masked_stack.bandNames())

In [12]:
# assess the model
# assessment
order = test.aggregate_array('class_name').distinct()
validated = test.classify(rf)
error_matrix = validated.errorMatrix("class_name", "classification", order=order)

In [13]:
from src.models.train_and_assess import format_error_matrix_for_export

fmt_error_matrix = format_error_matrix_for_export(error_matrix)

In [14]:
fmt_error_matrix

In [ ]:
# make prediction using the model
prediction = masked_stack.classify(rf)

In [ ]:
land_cover_colors = {
    "Bog": "#8B4513",          # Brown
    "Fen": "#808000",          # Olive Green
    "Marsh": "#008000",        # Medium Green
    "Salt marsh": "#9ACD32",   # Yellow-Green
    "Shallow water": "#ADD8E6",# Light Blue
    "Swamp": "#006400"         # Dark Green
}

In [ ]:
Map = geemap.Map()

Map.addLayer(prediction, {'palette': list(land_cover_colors.values()), 'min': 1, 'max': 6}, 'Prediction')
Map.addLayerControl()
Map.centerObject(aoi, zoom=8)

Map